In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import datetime

Get a largish dataset to demo (Wikidata9M dataset is the whole revision history of wikidata for the last ~9 years in the form of triples, the unzipped file is ~300MB in size)

In [ ]:
!wget https://github.com/rlafraie/Wikidata9M/raw/main/Wikidata9M.txt.bz2 && bzip2 -d  Wikidata9M.txt.bz2

--2022-08-06 20:41:02--  https://github.com/rlafraie/Wikidata9M/raw/main/Wikidata9M.txt.bz2
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rlafraie/Wikidata9M/main/Wikidata9M.txt.bz2 [following]
--2022-08-06 20:41:03--  https://raw.githubusercontent.com/rlafraie/Wikidata9M/main/Wikidata9M.txt.bz2
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63283797 (60M) [application/octet-stream]
Saving to: ‘Wikidata9M.txt.bz2’

Wikidata9M.txt.bz2  100%[===================>]  60.35M  95.8MB/s    in 0.6s    

2022-08-06 20:41:04 (95.8 MB/s) - ‘Wikidata9M.txt.bz2’ saved [63283797/63283797]

bzip2: Output fil

In [ ]:
!ls -l --block-size=M

total 423M
drwxr-xr-x 1 root root   1M Aug  3 20:21 sample_data
-rw-r--r-- 1 root root 363M Aug  6 18:41 Wikidata9M.txt
-rw-r--r-- 1 root root  61M Aug  6 20:41 Wikidata9M.txt.bz2


Separators for loading csv files: 

The file also has unconventional separator (it's a space, not tab or comma)

In [ ]:
!head Wikidata9M.txt

60 607 6 + 2013-02-04T19:22:48Z
220 463130 6 + 2013-02-04T19:25:58Z
648 228977 6 + 2013-02-04T19:35:04Z
220 38 17 + 2013-02-04T19:44:51Z
2135 16 17 + 2013-02-04T19:45:13Z
90 142 17 + 2013-02-04T19:46:08Z
90 256294 6 + 2013-02-04T19:46:48Z
1948 16 17 + 2013-02-04T19:47:33Z
84 145 17 + 2013-02-04T19:49:23Z
1384 30 17 + 2013-02-04T19:50:20Z


For example, if you try to use the tab as separator:

In [ ]:
try:
    df = pd.read_csv('Wikidata9M.txt', header=None, names =['subject', 'object', 'predicate', 'change', 'timestamp'], sep='\t')
except Exception as e:
    print(f'error: {e}')

In [ ]:
df.head()

,subject,object,predicate,change,timestamp
0,60 607 6 + 2013-02-04T19:22:48Z,NaN,NaN,NaN,NaN
1,220 463130 6 + 2013-02-04T19:25:58Z,NaN,NaN,NaN,NaN
2,648 228977 6 + 2013-02-04T19:35:04Z,NaN,NaN,NaN,NaN
3,220 38 17 + 2013-02-04T19:44:51Z,NaN,NaN,NaN,NaN
4,2135 16 17 + 2013-02-04T19:45:13Z,NaN,NaN,NaN,NaN


But you can use `\s+` to use a space as separator

In [ ]:
try:
    df = pd.read_csv('Wikidata9M.txt', header=None, names =['subject', 'object', 'predicate', 'change', 'timestamp'], sep='\s+')
except Exception as e:
    print(f'error: {e}')
df.head()

,subject,object,predicate,change,timestamp
0,60,607,6,+,2013-02-04T19:22:48Z
1,220,463130,6,+,2013-02-04T19:25:58Z
2,648,228977,6,+,2013-02-04T19:35:04Z
3,220,38,17,+,2013-02-04T19:44:51Z
4,2135,16,17,+,2013-02-04T19:45:13Z


If you know a column is a categorical value, you can shrink memory usage by converting it to categorical column (you can see that we have saved ~100MB) by doing this on all 4 of the categorical columns. As expected, columns with less unique values shrink more)

In [ ]:
# starting point
print(df.memory_usage(index=False))
print(f'total memory: {df.memory_usage(index=False).sum() / 1e6}MB')

subject      73809744
object       73809744
predicate    73809744
change       73809744
timestamp    73809744
dtype: int64
total memory: 369.04872MB


In [ ]:
df.nunique()

subject       432041
object        199149
predicate        407
change             2
timestamp    7877544
dtype: int64

In [ ]:
df['subject'] = df['subject'].astype('category')
print('converting subject to categorical')
print(f'total memory: {df.memory_usage(index=False).sum() / 1e6}MB')
print('-----------------------------------------------')

print('converting object to categorical')
df['object'] = df['object'].astype('category')
print(f'total memory: {df.memory_usage(index=False).sum() / 1e6}MB')
print('-----------------------------------------------')

print('converting predicate to categorical')
df['predicate'] = df['predicate'].astype('category')
print(f'total memory: {df.memory_usage(index=False).sum() / 1e6}MB')
print('-----------------------------------------------')

print('converting change to categorical')
df['change'] = df['change'].astype('category')
print(f'total memory: {df.memory_usage(index=False).sum() / 1e6}MB')
print('-----------------------------------------------')

converting subject to categorical
total memory: 352.508504MB
-----------------------------------------------
converting object to categorical
total memory: 321.423936MB
-----------------------------------------------
converting predicate to categorical
total memory: 266.086436MB
-----------------------------------------------
converting change to categorical
total memory: 201.503034MB
-----------------------------------------------


In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'] )
print(f'total memory: {df.memory_usage(index=False).sum() / 1e6}MB')

total memory: 201.503034MB


Named Aggregations: it lets us do many different operations after the dataframe has been grouped in a tidy way that is easy to read. You can also use a custom function which is really handy

In [ ]:
group_demo = df.groupby('change').agg(
    min_time=pd.NamedAgg(column="timestamp", aggfunc="min"),
    count_unique_subjects=pd.NamedAgg(column="subject", aggfunc=lambda x: len(x.drop_duplicates())),
)
group_demo

,min_time,count_unique_subjects
change,,
+,2013-02-04 19:22:48+00:00,432041
-,2013-02-04 20:09:09+00:00,274446


Axis: you can use 'columns'/'rows' to define (rather than remembering what axis=1/axis=0 means) 

axis is the direction along which to apply the compute (so for columsn you end up with something the same length as the starting)


In [ ]:
subset = df[['subject', 'predicate']].astype(int)
print('axis=columns:')
print(subset.mean(axis='columns'))
print('--------------------------------------')
print('axis=rows')
print(subset.mean(axis='rows'))

axis=columns:
0                33.0
1               113.0
2               327.0
3               118.5
4              1076.0
              ...    
9226213    28320937.0
9226214    28321217.0
9226215    28434570.5
9226216    28435602.0
9226217    21363256.5
Length: 9226218, dtype: float64
--------------------------------------
axis=rows
subject      6.619222e+06
predicate    3.937370e+02
dtype: float64


Pivot tables work for strings too, you don't need to have a numerical aggregate

In [7]:
date_sequence1 = [datetime.date(2020, 4, 13), datetime.date(2020, 7, 15),
       datetime.date(2020, 10, 15), datetime.date(2021, 1, 14),
       datetime.date(2021, 4, 13), datetime.date(2021, 7, 15),
       datetime.date(2021, 11, 4), datetime.date(2021, 11, 6)
       ]

date_sequence2 = [datetime.date(2020, 3, 13), datetime.date(2020, 6, 14),
       datetime.date(2020, 9, 14), datetime.date(2020, 12, 14),
       datetime.date(2021, 3, 13), datetime.date(2021, 6, 14),
       datetime.date(2021, 9, 14), datetime.date(2021, 12, 14)]
strings = [
    'actor', 'beekeeper', 'cat', 'dog', 'elf', 'hippo', 'ghost', 'tent'
]
example = pd.DataFrame(data = dict(
    date1 = date_sequence1,
    date2 = date_sequence2,
    text = strings)
)

pd.pivot_table(example,
               index='date1', 
               columns='date2', 
                         values= ['text'],aggfunc=lambda x: ', '.join(x)
                        ).fillna('')

text                                                         \
date2      2020-03-13 2020-06-14 2020-09-14 2020-12-14 2021-03-13 2021-06-14   
date1                                                                          
2020-04-13      actor                                                          
2020-07-15             beekeeper                                               
2020-10-15                              cat                                    
2021-01-14                                         dog                         
2021-04-13                                                    elf              
2021-07-15                                                             hippo   
2021-11-04                                                                     
2021-11-06                                                                     

                                  
date2      2021-09-14 2021-12-14  
date1                             
2020-04-13                        
2020-07-15                        
2020-10-15                        
2021-01-14                        
2021-04-13                        
2021-07-15                        
2021-11-04      ghost             
2021-11-06                  tent

Styling dataframes (more options here: https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)


In [ ]:
def custom_style(styler):
    styler.set_caption("Wikidata info")
    styler.format(formatter={('count_unique_subjects'): "{:,.0f}",
                           ('min_time'): lambda x: x.strftime('%d-%m-%Y'),
                          })
    styler.highlight_max(axis='rows', props='color:white; font-weight:bold; background-color:darkgreen;')
    # this only works for pandas v1.4+
    # styler.format_index(lambda v: 'delete' if v=='-' else 'add')

    return styler


group_demo.style.pipe(custom_style)

,min_time,count_unique_subjects
change,,
+,04-02-2013,"432,041"
-,04-02-2013,"274,446"
